In [46]:
import numpy as np
from sympy import *
import warnings
warnings.filterwarnings('ignore')

In [112]:
class BloomFilter:
    def __init__(self, **kwargs):
        #self.input_data = kwargs.get('input_data')
        self.N = kwargs.get('N')
        self.bit_array = [0]*self.N
        self.hash_funcs = []
        for k,v in kwargs.items():
            if k.startswith('k'):
                self.hash_funcs.append(v)
        self.generate_hash_functions()
        self.inserted_elements = []
                
    def generate_hash_functions(self):
        self.sym =symbols('x n')
        self.hash_funcs_expr = [parse_expr(e) for e in self.hash_funcs]
        
    def display_hash_functions(self):
        for i, expr in enumerate(self.hash_funcs_expr):
            display(Eq(Symbol(f'k{i+1}'), expr))
    
    def insert(self, input_data):
        print('Input data:')
        display(Matrix(input_data))
        
        print('\nBit Array:')
        display(Matrix(self.bit_array))
        
        for e in input_data:            
            print(f'\nInsert: {e}')
            self.inserted_elements.append(e)
            if isinstance(e, str):
                e = self.convert_string_to_numeric(e)
            display(Eq(Symbol('x'), e))  
            display(Eq(Symbol('n'), self.N))  
            for i, expr in enumerate(self.hash_funcs_expr):
                display(Eq(Symbol(f'k{i+1}'), expr))
                expr_val = expr.subs([(self.sym[0], e), (self.sym[1], self.N)])
                display(Eq(Symbol(f'k{i+1}'), expr_val))
                print(f'So we change bit {expr_val} in bit array to 1\n')
                self.bit_array[expr_val] = 1
            print('\nBit Array:')
            display(Matrix(self.bit_array))
            
    def convert_string_to_numeric(self, exp):
        print('\nConverting string value to numeric\n')
        val = 0
        for x in exp:
            val +=ord(x)
        return val
    
    def find_element(self, element):
        print(f'Searching element {element}\n')
        if isinstance(element, str):
            e = self.convert_string_to_numeric(element)
        display(Eq(Symbol('x'), element))
        display(Eq(Symbol('n'), self.N))
        hash_list = []
        bit_list = []
        for i, expr in enumerate(self.hash_funcs_expr):
                display(Eq(Symbol(f'k{i+1}'), expr))
                expr_val = expr.subs([(self.sym[0], element), (self.sym[1], self.N)])
                display(Eq(Symbol(f'k{i+1}'), expr_val))                
                hash_list.append(expr_val)
                bit_list.append(self.bit_array[expr_val])
        print('\nBit Array:')
        display(Matrix(self.bit_array))
        print(f"\nHash Function values are {','.join([str(s) for s in hash_list])}")
        print(f"\nCorresponding bits in bit array areare {','.join([str(s) for s in bit_list])}")
                                
        present = np.all(np.array(bit_list) == 1)
                                    
                                
        if present:
            print(f'\n{element} is probably in the table')
        else:
            print(f'\n{element} is definitely not in the table')
     
    def cal_false_positive_probability(self):
        m = len(self.bit_array)
        n = len(self.inserted_elements)
        k = len(self.hash_funcs)
        sym = symbols('m n k')
        fp_expr = parse_expr('(1-(1-1/m)**(k*n))**k')        
        display(Eq(Symbol('P'),fp_expr))
        print('\nWhere\n')
        display(Eq(Symbol('m'),m))
        display(Eq(Symbol('n'),n))
        display(Eq(Symbol('k'),k))
        fp_expr_val = fp_expr.subs([(sym[0], m), (sym[1], n), (sym[2], k)])
        display(Eq(Symbol('P'),N(fp_expr_val,4)))

In [113]:
bl = BloomFilter(N=7, k1 = '(13-(x % 13)) % n', k2 = '(3 + 5*x) % n')

In [114]:
bl.display_hash_functions()

Eq(k1, Mod(13 - Mod(x, 13), n))

Eq(k2, Mod(5*x + 3, n))

In [115]:
bl.insert([129, 479])

Input data:


Matrix([
[129],
[479]])


Bit Array:


Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0]])


Insert: 129


...

So we change bit 2 in bit array to 1



Eq(k2, Mod(5*x + 3, n))

Eq(k2, 4)

So we change bit 4 in bit array to 1


Bit Array:


Matrix([
[0],
[1],
[1],
[0],
[1],
[0],
[0]])

In [116]:
bl.find_element(23)

Searching element 23



Eq(x, 23)

Eq(n, 7)

Eq(k1, Mod(13 - Mod(x, 13), n))

Eq(k1, 3)

Eq(k2, Mod(5*x + 3, n))

Eq(k2, 6)


Bit Array:


Matrix([
[0],
[1],
[1],
[0],
[1],
[0],
[0]])


Hash Function values are 3,6

Corresponding bits in bit array areare 0,0

23 is definitely not in the table


In [117]:
bl.cal_false_positive_probability()

Eq(P, (1 - (1 - 1/m)**(k*n))**k)


Where



Eq(m, 7)

Eq(n, 2)

Eq(k, 2)

Eq(P, 0.2118)